In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from torch.distributed.pipelining import pipeline
from transformers import AutoTokenizer, AutoModel, pipeline
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import copy

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Load dataset

In [18]:
train_df = pd.read_csv('../data/train.txt', header=None, sep=";")
test_df = pd.read_csv('../data/test.txt', header=None, sep=";")
val_df = pd.read_csv('../data/validation.txt', header=None, sep=";")

In [19]:
train_df.head()

,0,1
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [20]:
train_df[1].value_counts()

1
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

We need to make a encoder for emotions

In [21]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df[1])
test_labels = label_encoder.transform(test_df[1])
val_labels = label_encoder.transform(val_df[1])

In [22]:
print(label_encoder.classes_)

['anger' 'fear' 'joy' 'love' 'sadness' 'surprise']


In [23]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")
model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_texts = train_df[0].tolist()
test_texts = test_df[0].tolist()
val_texts = val_df[0].tolist()

def get_embeddings(texts):
    batch_size = 32
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encodings = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
        encodings = {k: v.to(device) for k, v in encodings.items()}

        with torch.no_grad():
            outputs = model(**encodings)
            cls_embeddings = outputs.last_hidden_state[:, 0]  # CLS token

        all_embeddings.append(cls_embeddings.cpu())

    embeds = torch.cat(all_embeddings)
    return embeds

X_train_tensor = get_embeddings(train_texts)
X_test_tensor = get_embeddings(test_texts)
X_val_tensor = get_embeddings(val_texts)

### Dataset & DataLoader

In [24]:
y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)
y_val_tensor = torch.tensor(val_labels, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)

### Model

In [25]:
class FCNNModel(torch.nn.Module):
    def __init__(self, input_size=768, num_classes=6):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, 256)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(256, 128)
        self.relu = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(128, num_classes)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        out = self.fc3(x)
        return out

In [26]:
def train(model, train_loader, optimizer, criterion, device, epochs=10, val_loader=None, patience=5):
    model.to(device)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

        train_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.4f}", end="")

        if val_loader:
            val_loss, val_acc = evaluate(model, val_loader, criterion, device, return_metrics=True)
            print(f" | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            # Early stopping logic
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                best_model_wts = model.state_dict()
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                model.load_state_dict(best_model_wts)
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, "../models/best_fcnn_model.pth")
                break
        else:
            print()

In [27]:
def evaluate(model, data_loader, criterion, device, label_encoder=None, return_metrics=False):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total

    if return_metrics:
        avg_loss = total_loss / len(data_loader)
        return avg_loss, accuracy
    else:
        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_ if label_encoder else None))

In [28]:
criterion = torch.nn.CrossEntropyLoss()
model = FCNNModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train

In [29]:
train(model, train_loader, optimizer, criterion, device, epochs=30, val_loader=test_loader, patience=5)

Epoch 1/30 | Train Loss: 679.6614 | Train Acc: 0.4857 | Val Loss: 1.1983 | Val Acc: 0.5445
Epoch 2/30 | Train Loss: 608.1725 | Train Acc: 0.5433 | Val Loss: 1.1708 | Val Acc: 0.5615
Epoch 3/30 | Train Loss: 576.5800 | Train Acc: 0.5605 | Val Loss: 1.0964 | Val Acc: 0.5850
Epoch 4/30 | Train Loss: 559.8965 | Train Acc: 0.5748 | Val Loss: 1.0800 | Val Acc: 0.5935
Epoch 5/30 | Train Loss: 548.3778 | Train Acc: 0.5844 | Val Loss: 1.0746 | Val Acc: 0.5875
Epoch 6/30 | Train Loss: 535.3342 | Train Acc: 0.5940 | Val Loss: 1.0454 | Val Acc: 0.6020
Epoch 7/30 | Train Loss: 526.6202 | Train Acc: 0.6006 | Val Loss: 1.0407 | Val Acc: 0.6035
Epoch 8/30 | Train Loss: 521.5556 | Train Acc: 0.6059 | Val Loss: 1.0324 | Val Acc: 0.6130
Epoch 9/30 | Train Loss: 514.3788 | Train Acc: 0.6085 | Val Loss: 1.0227 | Val Acc: 0.6075
Epoch 10/30 | Train Loss: 507.1251 | Train Acc: 0.6176 | Val Loss: 1.0266 | Val Acc: 0.6145
Epoch 11/30 | Train Loss: 501.4812 | Train Acc: 0.6175 | Val Loss: 1.0296 | Val Acc: 0.60

In [31]:
model = FCNNModel()
model.load_state_dict(torch.load("../models/best_fcnn_model.pth"))
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
evaluate(model, val_loader, criterion, label_encoder=label_encoder, device=device, return_metrics=False)


Classification Report:
              precision    recall  f1-score   support

       anger       0.56      0.40      0.47       275
        fear       0.60      0.40      0.48       212
         joy       0.75      0.74      0.74       704
        love       0.53      0.28      0.37       178
     sadness       0.55      0.81      0.66       550
    surprise       0.45      0.36      0.40        81

    accuracy                           0.62      2000
   macro avg       0.57      0.50      0.52      2000
weighted avg       0.62      0.62      0.61      2000

